## <font color=009DD2> <span style="background-color: #A9C4C0"> Imports

In [50]:
import sys,json
import glob, os
sys.path.append('/media/miplab-nas2/Data3/BMPD/hc_project/analysis/code/')

from seed_to_voxels import Seed2voxels

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## <font color=009DD2> <span style="background-color: #A9C4C0"> Initialization

### <font color=009DD2> I Configuration & parameters </font>

In [51]:
with open('../config/config_NK.json') as config_file:
    config = json.load(config_file) # load config file

# Parameters (maybe we should move them to config?)
signal='raw'
#seed_names=['L-BA4']
seed_names=['PAM50_gm_C1C7_L','PAM50_gm_C1C7_R','PAM50_gm_C1C7_D','PAM50_gm_C1C7_V']

seed_structure = 'spinalcord'
target_structure='brain' # Define structure to use as target (brain or spinal cord)
seed_folder = '/C1toC7/' # Name of folder containing seed tcs, etc.

### <font color=009DD2> II Select files </font>

In [52]:
# One target per subject
data_target=[];ts_target_txt=[]
for subject_name in config['list_subjects']:
    if target_structure == 'spinalcord':
        data_target.append(config['coreg_dir'] + '/sub-' + subject_name + '/' + target_structure + '/sub-' + subject_name + '_task-rest_bold_stc_sc_' + config['coreg_tag'][target_structure] + '.nii')
    elif  target_structure == 'brain':
        data_target.append(config['coreg_dir'] + '/sub-' + subject_name + '/' + target_structure + '/sub-' + subject_name + '_task-rest_bold_stc_brain_' + config['coreg_tag'][target_structure] + '.nii')
    ts_target_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/' +target_structure+'_all/timeseries/sub-' + subject_name + '_' + target_structure + '_all_timeseries')

# Create directories if needed
if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/' +target_structure+'_all'):
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/' +target_structure+'_all')
if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/' +target_structure+'_all/timeseries/'):
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/' +target_structure+'_all/timeseries/') # folder to store timeseries extraction

# We take the mask of brain or spinal cord as needed for the target 
mask_target=config['main_dir'] + config['masks'][target_structure]

# But possible to have multiple seeds (so create a dictionary to store list of each seed)
data_seed={}; ts_seed_txt={}; mask_seed={}
for seed_name in seed_names:
    data_seed[seed_name] = []
    ts_seed_txt[seed_name] = []
    
    # Create directory for seed if it doesn't exist
    if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + seed_folder):
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + seed_folder) 
    if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + seed_folder + seed_name):
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + seed_folder + seed_name)

    if seed_structure == 'spinalcord':    
        mask_seed[seed_name] = glob.glob(config['main_dir'] + config['templates']['sc_axialdiv_path'] + seed_name + '*.gz')[0]
    elif seed_structure == 'brain':  
        mask_seed[seed_name] = '/media/miplab-nas2/Data3/BMPD/hc_project/seed_to_voxels/L-BA4/L-BA4.nii.gz'
    
    for subject_name in config['list_subjects']:
        if seed_structure == 'spinalcord':    
            data_seed[seed_name].append(config['coreg_dir'] + 'sub-'+ subject_name + '/' + seed_structure + '/sub-' + subject_name + '_task-rest_bold_stc_sc_' + config['coreg_tag'][seed_structure] + '.nii')
        elif seed_structure == 'brain':  
            data_seed[seed_name].append(config['coreg_dir'] + 'sub-'+ subject_name + '/' + seed_structure + '/sub-' + subject_name + '_task-rest_bold_stc_brain_' + config['coreg_tag'][seed_structure] + '.nii')
        
        # Here, spinal cord is going to be smoothed, hence the tag _s3x3x6 to avoid confusion
        ts_seed_txt[seed_name].append(config['main_dir'] + config['seed2vox_dir'] + seed_folder + seed_name+'/timeseries/sub-' + subject_name + '_' + seed_name + '_timeseries')
    
    # Create folder for timeseries
    if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + seed_folder + seed_name + '/timeseries/'):
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + seed_folder + seed_name + '/timeseries/')

    # Create folder for brain maps
    if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + seed_folder + seed_name + '/' +  target_structure + '_fc_maps/'):
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] + seed_folder + seed_name + '/' +  target_structure + '_fc_maps/')





In [53]:
seed2voxels=Seed2voxels(config,signal,seed_names,target_structure) # initialize the function

## <font color=009DD2> <span style="background-color: #A9C4C0"> Data extraction


### <font color=009DD2> I. Time series extraction - Seed </font>

In [46]:
# Dictionary with seed names
# Contains one array per subject

# !! Here, no smoothing in seed

seed_timeseries={};seed_timeseries_mean={};seed_timeseries_pc1={};
for seed_name in seed_names:
    seed_timeseries[seed_name]=[]
    seed_timeseries_mean[seed_name]=[]
    seed_timeseries_pc1[seed_name]=[]
    (seed_timeseries[seed_name],seed_timeseries_mean[seed_name],seed_timeseries_pc1[seed_name])=seed2voxels.extract_data(img=data_seed[seed_name], mask=mask_seed[seed_name],timeseries_txt=ts_seed_txt[seed_name],run="extract",n_jobs=8) # Load or extract timecourses in seed

### <font color=009DD2> II. Time series extraction - Target </font>

In [47]:
(target_timeseries,target_timeseries_mean,target_timeseries_pc1)=seed2voxels.extract_data(img=data_target, smoothing_fwhm=[6,6,6], mask=mask_target, timeseries_txt=ts_target_txt,
                run="extract",n_jobs=8) # Load voxelwise brain signals as target

## <font color=009DD2> <span style="background-color: #A9C4C0"> Correlation analysis

In [66]:
output_img={};
for seed_name in seed_names:
    output_img[seed_name]=[]
    output_img[seed_name]=config['main_dir'] + config['seed2vox_dir'] + seed_folder + '/'+seed_name+'/'+target_structure+'_fc_maps/' + str(len(config['list_subjects'])) + 'subjects_' + seed_name
    seed2voxels.correlation_maps(seed_timeseries_mean[seed_name],target_timeseries,mask=mask_target,output_img=output_img[seed_name],Fisher=True,save_maps=True,n_jobs=8)

## <font color=009DD2> <span style="background-color: #A9C4C0"> Statistics (non-parametric permutation testing)

### <font color=009DD2> Mean correlations </font>

In [49]:
for seed_name in seed_names:
    run_string = 'randomise -i ' + output_img[seed_name] + '_zcorr.nii -o ' + output_img[seed_name] + ' -1 -T -m ' + mask_target + ' -n 200'
    os.system(run_string)

randomise options: -i /media/miplab-nas2/Data3/BMPD/hc_project//seed_to_voxels///PAM50_gm_C1C7_L/brain_fc_maps/22subjects_PAM50_gm_C1C7_L_zcorr.nii -o /media/miplab-nas2/Data3/BMPD/hc_project//seed_to_voxels///PAM50_gm_C1C7_L/brain_fc_maps/22subjects_PAM50_gm_C1C7_L -1 -T -m /media/miplab-nas2/Data3/BMPD/hc_project//templates/MNI_hc_project/MNI_T1w_GM_AverageHCs_3mm_bin_corrected.nii -n 200 
Loading Data: 
Data loaded
One-sample design detected; sign-flipping instead of permuting.
4.1943e+06 sign-flips required for exhaustive test of t-test 1
Doing 200 random permutations
Starting permutation 1 (Unpermuted data)
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
Starting permutation 6
Starting permutation 7
Starting permutation 8
Starting permutation 9
Starting permutation 10
Starting permutation 11
Starting permutation 12
Starting permutation 13
Starting permutation 14
Starting permutation 15
Starting permutation 16
Starting permutation 17
Star

Starting permutation 83
Starting permutation 84
Starting permutation 85
Starting permutation 86
Starting permutation 87
Starting permutation 88
Starting permutation 89
Starting permutation 90
Starting permutation 91
Starting permutation 92
Starting permutation 93
Starting permutation 94
Starting permutation 95
Starting permutation 96
Starting permutation 97
Starting permutation 98
Starting permutation 99
Starting permutation 100
Starting permutation 101
Starting permutation 102
Starting permutation 103
Starting permutation 104
Starting permutation 105
Starting permutation 106
Starting permutation 107
Starting permutation 108
Starting permutation 109
Starting permutation 110
Starting permutation 111
Starting permutation 112
Starting permutation 113
Starting permutation 114
Starting permutation 115
Starting permutation 116
Starting permutation 117
Starting permutation 118
Starting permutation 119
Starting permutation 120
Starting permutation 121
Starting permutation 122
Starting permutat

Starting permutation 186
Starting permutation 187
Starting permutation 188
Starting permutation 189
Starting permutation 190
Starting permutation 191
Starting permutation 192
Starting permutation 193
Starting permutation 194
Starting permutation 195
Starting permutation 196
Starting permutation 197
Starting permutation 198
Starting permutation 199
Starting permutation 200
Critical Value for: /media/miplab-nas2/Data3/BMPD/hc_project//seed_to_voxels///PAM50_gm_C1C7_D/brain_fc_maps/22subjects_PAM50_gm_C1C7_D_tfce_tstat1 is: 6384.76
Finished, exiting.
randomise options: -i /media/miplab-nas2/Data3/BMPD/hc_project//seed_to_voxels///PAM50_gm_C1C7_V/brain_fc_maps/22subjects_PAM50_gm_C1C7_V_zcorr.nii -o /media/miplab-nas2/Data3/BMPD/hc_project//seed_to_voxels///PAM50_gm_C1C7_V/brain_fc_maps/22subjects_PAM50_gm_C1C7_V -1 -T -m /media/miplab-nas2/Data3/BMPD/hc_project//templates/MNI_hc_project/MNI_T1w_GM_AverageHCs_3mm_bin_corrected.nii -n 200 
Loading Data: 
Data loaded
One-sample design detect

### <font color=009DD2> Contrasts </font>

In [72]:
seeds_of_interest = ['PAM50_gm_C1C7_L','PAM50_gm_C1C7_R']
for t_ix,t in enumerate(seeds_of_interest):
    to_contrast[t_ix] = seeds_of_interest[t_ix].split('_')[-1]
mean_dir = config['main_dir'] + config['seed2vox_dir'] + seed_folder 
contrast_dir = config['main_dir'] + config['seed2vox_dir'] + seed_folder + 'PAM50_gm_C1C7_' + str(to_contrast[0]) + str(to_contrast[1] + '/')
contrast_file = 'PAM50_gm_C1C7_' + str(to_contrast[0]) + str(to_contrast[1]) + '.nii.gz'
# Create folder for contrast if needed
if not os.path.exists(contrast_dir):
    os.mkdir(contrast_dir) 
# Merge files if needed 
if not os.path.exists(contrast_dir + contrast_file):
    concat_string = 'fslmerge -t ' + contrast_dir + contrast_file + ' ' + output_img[seeds_of_interest[0]] + '_zcorr ' + output_img[seeds_of_interest[1]] + '_zcorr'
    os.system(concat_string)
run_string = 'randomise -i ' + contrast_dir + contrast_file + ' -o ' + contrast_dir + contrast_file + ' -d ' + config['main_dir'] + config['seed2vox_dir'] + 'design_diff_22sub.mat' + ' -t ' + config['main_dir'] + config['seed2vox_dir'] + 'design_diff.con' + ' -e ' + config['main_dir'] + config['seed2vox_dir'] + 'design_diff_22sub.grp -T -m ' + mask_target + ' -n 200'
os.system(run_string)

randomise options: -i /media/miplab-nas2/Data3/BMPD/hc_project//seed_to_voxels//C1toC7/PAM50_gm_C1C7_LR/PAM50_gm_C1C7_LR.nii.gz -o /media/miplab-nas2/Data3/BMPD/hc_project//seed_to_voxels//C1toC7/PAM50_gm_C1C7_LR/PAM50_gm_C1C7_LR.nii.gz -d /media/miplab-nas2/Data3/BMPD/hc_project//seed_to_voxels/design_diff_22sub.mat -t /media/miplab-nas2/Data3/BMPD/hc_project//seed_to_voxels/design_diff.con -e /media/miplab-nas2/Data3/BMPD/hc_project//seed_to_voxels/design_diff_22sub.grp -T -m /media/miplab-nas2/Data3/BMPD/hc_project//templates/MNI_hc_project/MNI_T1w_GM_AverageHCs_3mm_bin_corrected.nii -n 200 
Loading Data: 
Data loaded
3.14573e+06 permutations required for exhaustive test of t-test 1
Doing 200 random permutations
Starting permutation 1 (Unpermuted data)
Starting permutation 2
Starting permutation 3
Starting permutation 4
Starting permutation 5
Starting permutation 6
Starting permutation 7
Starting permutation 8
Starting permutation 9
Starting permutation 10
Starting permutation 11
St

Starting permutation 98
Starting permutation 99
Starting permutation 100
Starting permutation 101
Starting permutation 102
Starting permutation 103
Starting permutation 104
Starting permutation 105
Starting permutation 106
Starting permutation 107
Starting permutation 108
Starting permutation 109
Starting permutation 110
Starting permutation 111
Starting permutation 112
Starting permutation 113
Starting permutation 114
Starting permutation 115
Starting permutation 116
Starting permutation 117
Starting permutation 118
Starting permutation 119
Starting permutation 120
Starting permutation 121
Starting permutation 122
Starting permutation 123
Starting permutation 124
Starting permutation 125
Starting permutation 126
Starting permutation 127
Starting permutation 128
Starting permutation 129
Starting permutation 130
Starting permutation 131
Starting permutation 132
Starting permutation 133
Starting permutation 134
Starting permutation 135
Starting permutation 136
Starting permutation 137
St

0